# 使用 Scikit-Learn 的線性迴歸

有一套開源且可商用的機器學習工具包，名為 [scikit-learn](https://scikit-learn.org/stable/index.html)。

這個工具包已經實作了本課程中你會接觸到的許多演算法。

<br>

## 目標
在這個實驗中，你將：
- 使用 scikit-learn 來實作利用梯度下降（Gradient Descent）的線性迴歸

<br>

## 工具
你會使用來自 scikit-learn 的函式，以及 matplotlib 和 NumPy。 

In [ ]:
import sys, os
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

#region 匯入資料
def find_repo_root(marker="README.md"):
    cur = Path.cwd()
    while cur != cur.parent:  # 防止無限迴圈，到達檔案系統根目錄就停
        if (cur / marker).exists():
            return cur
        cur = cur.parent
    return None

def import_data_from_github():
    import urllib.request, shutil
    
    def isRunningInColab() -> bool:
        return "google.colab" in sys.modules

    def isRunningInJupyterLab() -> bool:
        try:
            import jupyterlab
            return True
        except ImportError:
            return False
        
    def detect_env():
        from IPython import get_ipython
        if isRunningInColab():
            return "Colab"
        elif isRunningInJupyterLab():
            return "JupyterLab"
        elif "notebook" in str(type(get_ipython())).lower():
            return "Jupyter Notebook"
        else:
            return "Unknown"
        
    def get_utils_dir(env): 
        if env == "Colab": 
            if "/content" not in sys.path:
                sys.path.insert(0, "/content")
            return "/content/utils"
        else:
            return Path.cwd() / "utils"

    def get_data_dir(env): 
        if env == "Colab": 
            if "/content" not in sys.path:
                sys.path.insert(0, "/content")
            return "/content/data"
        else:
            return Path.cwd() / "data"

    env = detect_env()
    UTILS_DIR = get_utils_dir(env)
    DATA_DIR = get_data_dir(env)
    REPO_DIR = "Machine-Learning-Lab"

    #shutil.rmtree(UTILS_DIR, ignore_errors=True)
    os.makedirs(UTILS_DIR, exist_ok=True)

    #shutil.rmtree(DATA_DIR, ignore_errors=True)
    os.makedirs(DATA_DIR, exist_ok=True)

    BASE = f"https://raw.githubusercontent.com/mz038197/{REPO_DIR}/main"
    urllib.request.urlretrieve(f"{BASE}/utils/lab_utils_multi.py", f"{UTILS_DIR}/lab_utils_multi.py")
    urllib.request.urlretrieve(f"{BASE}/utils/lab_utils_common.py", f"{UTILS_DIR}/lab_utils_common.py")
    urllib.request.urlretrieve(f"{BASE}/utils/deeplearning.mplstyle", f"{UTILS_DIR}/deeplearning.mplstyle")
    urllib.request.urlretrieve(f"{BASE}/data/houses.txt", f"{DATA_DIR}/houses.txt")

repo_root = find_repo_root()

if repo_root is None:
    import_data_from_github()
    repo_root = Path.cwd()
    
os.chdir(repo_root)
print(f"✅ 切換工作目錄至 {Path.cwd()}")
sys.path.append(str(repo_root)) if str(repo_root) not in sys.path else None
print(f"✅ 加入到系統路徑")

from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from utils.lab_utils_multi import  load_house_data
from utils.lab_utils_common import dlc
np.set_printoptions(precision=2) 
plt.style.use('utils/deeplearning.mplstyle')

print("✅ 匯入模組及設定繪圖樣式")
#endregion 匯入資料


<br>

# 梯度下降（Gradient Descent）
Scikit-learn 提供了一個梯度下降的迴歸模型 [sklearn.linear_model.SGDRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html#examples-using-sklearn-linear-model-sgdregressor)。就像你先前實作的梯度下降一樣，這個模型在輸入特徵正規化之後效果最佳。[sklearn.preprocessing.StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler) 會對特徵做 z-score 正規化，就像先前的實驗一樣。這裡也把它稱為「標準化分數（standard score）」。

### 載入資料集

In [ ]:
X_train, y_train = load_house_data()
X_features = ['size(sqft)','bedrooms','floors','age']

### 對訓練資料做縮放／正規化

In [ ]:
scaler = StandardScaler()
X_norm = scaler.fit_transform(X_train)
print(f"Peak to Peak range by column in Raw        X:{np.ptp(X_train,axis=0)}")   
print(f"Peak to Peak range by column in Normalized X:{np.ptp(X_norm,axis=0)}")

### 建立並訓練迴歸模型

In [ ]:
sgdr = SGDRegressor(max_iter=1000)
sgdr.fit(X_norm, y_train)
print(sgdr)
print(f"number of iterations completed: {sgdr.n_iter_}, number of weight updates: {sgdr.t_}")

### 檢視參數
請注意，這些參數是對應到**經過正規化**的輸入資料。這裡訓練出來的參數和你在前一個使用同一份資料的實驗中得到的參數非常接近。

In [ ]:
b_norm = sgdr.intercept_
w_norm = sgdr.coef_
print(f"model parameters:                   w: {w_norm}, b:{b_norm}")
print( "model parameters from previous lab: w: [110.56 -21.27 -32.71 -37.97], b: 363.16")

### 進行預測
對訓練資料做預測。請同時使用 `predict` 函式，以及利用參數 \(w\) 和 \(b\) 手動計算。

In [ ]:
# make a prediction using sgdr.predict()
y_pred_sgd = sgdr.predict(X_norm)
# make a prediction using w,b. 
y_pred = np.dot(X_norm, w_norm) + b_norm  
print(f"prediction using np.dot() and sgdr.predict match: {(y_pred == y_pred_sgd).all()}")

print(f"Prediction on training set:\n{y_pred[:4]}" )
print(f"Target values \n{y_train[:4]}")

### 繪製結果
我們來將預測值和目標值畫在同一張圖上比較。

In [ ]:
# plot predictions and targets vs original features    
fig,ax=plt.subplots(1,4,figsize=(12,3),sharey=True)
for i in range(len(ax)):
    ax[i].scatter(X_train[:,i],y_train, label = 'target')
    ax[i].set_xlabel(X_features[i])
    ax[i].scatter(X_train[:,i],y_pred,color=dlc["dlorange"], label = 'predict')
ax[0].set_ylabel("Price"); ax[0].legend();
fig.suptitle("target versus prediction using z-score normalized model")
plt.show()

## 恭喜！
在這個實驗中，你已經：
- 使用了開源的機器學習工具包 scikit-learn
- 透過該工具包中的功能，完成了結合梯度下降與特徵正規化的線性迴歸實作